In [ ]:
!chmod u+x predict_all.sh

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
!./train_all.sh

Using TensorFlow backend.
***************************** FOLD 0 *****************************
Training on 3127 samples
Validating on 793 samples
['unet_resnet_50_exp_0_144_192_snapshot_50_epochs_vp_1', 'fold_0.hdf5']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 192, 192, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 192, 192, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 198, 198, 3)  0           bn_data[0][0]                    
___________________________________________________________________________________________

No weights passed, training from scratch
Epoch 1/50
260/260 [==============================] - 123s 475ms/step - loss: 0.5779 - Kaggle_IoU_Precision: 0.2379 - val_loss: 0.3834 - val_Kaggle_IoU_Precision: 0.4516
Epoch 2/50
260/260 [==============================] - 114s 437ms/step - loss: 0.4737 - Kaggle_IoU_Precision: 0.4853 - val_loss: 0.3005 - val_Kaggle_IoU_Precision: 0.7538
Epoch 3/50
260/260 [==============================] - 114s 437ms/step - loss: 0.4442 - Kaggle_IoU_Precision: 0.5814 - val_loss: 0.3030 - val_Kaggle_IoU_Precision: 0.7334
Epoch 4/50
260/260 [==============================] - 114s 437ms/step - loss: 0.4348 - Kaggle_IoU_Precision: 0.6047 - val_loss: 0.2440 - val_Kaggle_IoU_Precision: 0.7702
Epoch 5/50
 44/260 [====>.........................] - ETA: 1:29 - loss: 0.4318 - Kaggle_IoU_Precision: 0.5967

In [ ]:
!./evaluate_all.sh

In [ ]:
!./predict_all.sh

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
%matplotlib inline
import cv2
import pandas as pd
import numpy as np
import os

%reload_ext autoreload
%autoreload 2

import keras
from keras import backend as K
import gc
from keras.optimizers import Adam, RMSprop, SGD

from utils import predict_test, evaluate, ensemble, ThreadsafeIter, classification_predict_test
from datasets.generators import SegmentationDataGenerator, ClassificationDataGenerator

from params import args
from callbacks import get_callback
from augmentations import get_augmentations

from models.models import get_model

from losses import *

In [ ]:

train = pd.read_csv(os.path.join(args.data_root,'train_proc_v2.csv'))
test = pd.read_csv(os.path.join(args.data_root,'sample_submission.csv'))

In [ ]:
# Cyclic LR
# T = nb_epochs
# M = nb_snapshots
# alpha_zero = init_lr

T = 200
M = 4
alpha_zero = 0.0001

import numpy as np
t = 199

cos_inner = np.pi * (t % (T // M))  # t - 1 is used when t has 1-based indexing.
cos_inner /= T // M
cos_out = np.cos(cos_inner) + 1
float(alpha_zero / 2 * cos_out)

In [ ]:
train.head()

In [ ]:
img = cv2.imread(os.path.join(args.images_dir,'{}.png'.format('88839f49f9')), cv2.IMREAD_COLOR)

In [ ]:
plt.imshow(img)

In [ ]:
plt.imshow(img_pad)

In [ ]:
from albumentations import *
#augmentation = PadIfNeeded(min_height=128, min_width=128, p=1.0, border_mode=0) 
augmentation = Compose([
            #RandomRotate90(p=1),
                HorizontalFlip(p=.5),
                RandomBrightness(p=.2,limit=0.2),
                RandomContrast(p=.1,limit=0.2),
                ShiftScaleRotate(shift_limit=0.1625, scale_limit=0., rotate_limit=0, p=1),
                RandomCrop(height=96, width=96,p=1)
            ], p=1)

data = {"image": img}
augmented = augmentation(**data)
img_pad = augmented["image"]

# from albumentations import PadIfNeeded
# augmentation = PadIfNeeded(min_height=128, min_width=128, p=1.0, border_mode=4) 
# data = {"image": img}
# augmented = augmentation(**data)
# img_pad = augmented["image"]

In [ ]:
for network in args.network.split(','):
    folds = [int(f) for f in args.fold.split(',')]
    for fold in folds:
        K.clear_session()
        print('***************************** FOLD {} *****************************'.format(fold))

        MODEL_PATH = os.path.join(args.models_dir,network+args.alias)

#         if fold == 0:
#             if os.path.isdir(MODEL_PATH):
#                 raise ValueError('Such Model already exists')
#             os.system("mkdir {}".format(MODEL_PATH))
#             os.system("cp params.py {}".format(MODEL_PATH))

        df_train = train[train.fold != fold].copy().reset_index(drop=True)
        df_valid = train[train.fold == fold].copy().reset_index(drop=True)

        ids_train, ids_valid = df_train[df_train.unique_pixels>1].id.values, df_valid.id.values

        # Fold 0
        # Training on 3127 samples
        # Validating on 810 samples
        # ids_train, ids_valid = df_train[(df_train.unique_pixels>1)&(~df_train.id.isin(bad_masks))].id.values, df_valid.id.values


        print('Training on {} samples'.format(ids_train.shape[0]))
        print('Validating on {} samples'.format(ids_valid.shape[0]))

        # Initialize Model
        weights_path = os.path.join(MODEL_PATH,'fold_{fold}.hdf5'.format(fold=fold))
        #weights_path = os.path.join(MODEL_PATH,'fold_{fold}'.format(fold=fold))
        
        print(weights_path.split('/')[-2:])


        model, preprocess = get_model(network, input_shape=(args.input_size, args.input_size, 3), train_base=True)
        print(model.summary())
        model.compile(optimizer=RMSprop(lr=args.learning_rate), loss=make_loss(args.loss_function),
                          metrics=[Kaggle_IoU_Precision])

        if args.weights is None:
            print('No weights passed, training from scratch')
        else:
            wp = args.weights.format(fold)
            print('Loading weights from {}'.format(wp))
            model.load_weights(wp, by_name=True)

        augs = get_augmentations(args.augmentation_name, p=args.augmentation_prob, input_shape=(args.input_size, args.input_size, 3))


        dg = SegmentationDataGenerator(input_shape=(args.input_size, args.input_size), batch_size = args.batch_size, augs = augs,
                          preprocess = preprocess)

        train_generator = dg.train_batch_generator(ids_train)
        validation_generator = dg.evaluation_batch_generator(ids_valid)

        callbacks = get_callback(args.callback, weights_path=weights_path,
                                fold = fold)

#         # Fit the model with Generators:
#         model.fit_generator(generator=ThreadsafeIter(train_generator),
#                         steps_per_epoch=ids_train.shape[0] // args.batch_size * 2,
#                         epochs=args.epochs,
#                         callbacks=callbacks,
#                         validation_data=ThreadsafeIter(validation_generator),
#                         validation_steps=np.ceil(ids_valid.shape[0] / args.batch_size),
#                         workers=12)

        
#         dir_path = os.path.join(MODEL_PATH,'oof')
#         os.system("mkdir {}".format(dir_path))
#         best_res = 0
#         best_weights_path = ''
#         for w in sorted(os.listdir(MODEL_PATH))[-13:-2]:
#             print(w)
#             weights_path = os.path.join(MODEL_PATH,w)
#             model.load_weights(weights_path,by_name=False)

#             # SAVE OOF PREDICTIONS
#             pred = predict_test(model=model,
#                             preds_path=dir_path,
#                             oof=True,
#                             ids=ids_valid,
#                             batch_size=args.batch_size*2,
#                             thr=0.5,
#                             TTA='',
#                             preprocess=preprocess)
#             res = evaluate([MODEL_PATH], train[train.fold.isin([fold])].id.values, 0.5, classification='')
#             print(np.mean(res['iout']))
#             if np.mean(res['iout']) > best_res:
#                 best_res = np.mean(res['iout'])
#                 best_weights_path = weights_path
        
#         print(best_weights_path)
#         print(best_res)
        
#         best_weights_path = weights_path
#         model.load_weights(best_weights_path)

#         # SAVE OOF PREDICTIONS
#         dir_path = os.path.join(MODEL_PATH,'oof')
#         os.system("mkdir {}".format(dir_path))
#         pred = predict_test(model=model,
#                         preds_path=dir_path,
#                         oof=True,
#                         ids=ids_valid,
#                         batch_size=args.batch_size*2,
#                         thr=0.5,
#                         TTA='flip',
#                         preprocess=preprocess)

        # SAVE TEST PREDICTIONS
#         dir_path = os.path.join(MODEL_PATH,'fold_{}'.format(fold))
#         os.system("mkdir {}".format(dir_path))
#         pred = predict_test(model=model,
#                         preds_path=dir_path,
#                         oof=False,
#                         ids=test.id.values,
#                         batch_size=args.batch_size*2,
#                         thr=0.5,
#                         TTA='flip',
#                         preprocess=preprocess)

        # K.clear_session()

In [ ]:
200*114/60/60

In [ ]:
args.alias

In [ ]:
577536

In [ ]:
MODEL_PATH

In [ ]:
models = [
#     'unet_resnet_152_valid_plus_resize_202_pad_256_bs_12_corr',
#           'unet_resnet_152_valid_plus_resize_202_pad_256_bs_12_corr',
#           'unet_resnet_152_finetuning_v2_reduce_lr_flip',
#           'unet_resnet_50unet_resnet_50_exp_0_144_192_new_scheduler_200_epochs',
#           'unet_resnet_50unet_resnet_50_exp_0_144_192_new_scheduler_200_epochs_bce_jacard_dice',
#           'resnet50_fpn_96_soft_early_stopping',
#           'resnet50_fpn_old',
#           'unet_resnet_34_valid_plus_resize_202_pad_256_bs_24',
#           'unet_resnet_152_exp_0_202_256_scheduler_150_epochs_finetune',
#           'unet_resnet_152_exp_0_202_256_scheduler_150_epochs_finetune'
    'unet_resnet_50_exp_0_160_160_snapshot_50_epochs',
    #'unet_resnet_50_exp_0_224_224_snapshot_50_epochs',
    #'unet_resnet_50_exp_0_192_192_snapshot_50_epochs'
         ]
model_pathes = ['/home/branding_images/salt/'+x for x in models]
res = evaluate(model_pathes, train[train.fold.isin([0])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
0.88037 / 0.91051 / 0.89145

In [ ]:
!./evaluate_all.sh

In [ ]:
unet_resnet_50_exp_0_128_128_snapshot_50_epochs
best: 0.84 / 0.88259 / 0.85977 + 
snapshot: 0.84012 / 0.88206 / 0.85958
    
unet_resnet_50_exp_0_160_160_snapshot_50_epochs
best: 0.85815 / 0.89563 / 0.8748 + 
snapshot: 0.84951 / 0.88725 / 0.8658
    
unet_resnet_50_exp_0_192_192_snapshot_50_epochs
best: 0.84938 / 0.88747 / 0.86692 +
snapshot: 0.84605 / 0.8844 / 0.86327
    
unet_resnet_50_exp_0_224_224_snapshot_50_epochs
best: 0.84568 / 0.8805 / 0.85981 +
snapshot: 0.84753 / 0.88444 / 0.86321 +
    
unet_resnet_50_exp_0_256_256_snapshot_50_epochs
best: 0.83395 / 0.87394 / 0.8526 +
snapshot: 0.83037 / 0.86933 / 0.84849 +
    
unet_resnet_50_exp_0_144_192_snapshot_50_epochs
best: 0.85827 / 0.89389 / 0.87291 +
snapshot: 0.85457 / 0.89015 / 0.86881 +
    
unet_resnet_50_exp_0_160_192_snapshot_50_epochs
best: 0.85568 / 0.89174 / 0.86991 +
snapshot: 0.85494 / 0.89127 / 0.86885 +
    
unet_resnet_50_exp_0_176_192_snapshot_50_epochs
best: 0.85815 / 0.89138 / 0.87119 +
snapshot: 0.85025 / 0.88615 / 0.86518 -
    
unet_resnet_50_exp_0_144_224_snapshot_50_epochs
best: 0.85198 / 0.88523 / 0.8643 +
snapshot: 0.85494 / 0.89099 / 0.86941 -
    
unet_resnet_50_exp_0_192_224_snapshot_50_epochs
best: 0.85914 / 0.89893 / 0.87727 +
snapshot: 0.85864 / 0.89403 / 0.87285 -
    
unet_resnet_50_exp_0_202_256_snapshot_50_epochs
best: 0.84901 / 0.88852 / 0.86643 +
snapshot: 0.85136 / 0.8927 / 0.87084 -
    
unet_resnet_152_exp_0_192_224_snapshot_50_epochs
best: 0.86395 / 0.89585 / 0.8749 +
snapshot: 0.86321 / 0.89516 / 0.87405 -
    
unet_resnet_50_exp_0_144_192_snapshot_50_epochs_lr_00002_v2
best: 0.85519 / 0.88991 / 0.86846
snapshot: 0.85815 / 0.89012 / 0.86858

    
0.87284 +

In [ ]:
MODEL_PATH = '/home/branding_images/salt/unet_resnet_50_exp_0_144_192_snapshot_50_epochs_lr_00002_v2'

In [ ]:
from utils import evaluate,evaluate_exp
res = evaluate([MODEL_PATH], train[train.fold.isin([0])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))


In [ ]:
args.input_size

In [ ]:
dir_path = os.path.join(MODEL_PATH,'oof')
#w='snapshot-1.h5'
w='fold_0.hdf5'
weights_path = os.path.join(MODEL_PATH,w)
model.load_weights(weights_path,by_name=False)

# SAVE OOF PREDICTIONS
pred = predict_test(model=model,
                preds_path=dir_path,
                oof=True,
                ids=ids_valid,
                batch_size=args.batch_size*2,
                thr=0.5,
                TTA='flip',
                preprocess=preprocess)
res = evaluate([MODEL_PATH], train[train.fold.isin([0])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))


In [ ]:
NEEW!
**************************************
unet_resnet_50_exp_0_resize_144_pad_192
0.83531 / 0.88551 / 0.86118

resnet50_fpn_exp_0_resize_144_pad_192
0.84185 / 0.88208 / 0.85874   
**************************************

7 and 15 for early stopping

# NO TTA
? unet_resnet_50_exp_0_resize_144_pad_192
? 0.83531 / 0.88551 / 0.86118

unet_resnet_50_exp_0_144_192_new
0.85951 / 0.89916 / 0.87709
unet_resnet_50unet_resnet_50_exp_0_144_192_new_again
0.85568 / 0.89098 / 0.86968

- resnet50_fpn_exp_0_144_192_new
0.85667 / 0.89677 / 0.87466

- unet_resnet_50_exp_0_144_192_rotate_limit_15
0.85753 / 0.89051 / 0.87015

- unet_resnet_50_exp_0_144_192_no_dropout
0.85173 / 0.89125 / 0.86898

++ unet_resnet_50unet_resnet_50_exp_0_144_192_new_scheduler_200_epochs
0.86519 / 0.89751 / 0.87723

-? unet_resnet_50unet_resnet_50_exp_0_144_192_new_scheduler_200_epochs_bce_jacard_dice
0.86198 / 0.89741 / 0.87645

+? unet_resnet_50unet_resnet_50_exp_0_144_192_new_scheduler_200_epochs_finetune
0.86654 / 0.90456 / 0.88377

run ResNet152_fpn over the night

In [ ]:
NEEW!

unet_resnet_34_exp_0_resize_144_pad_192_cyclic_lr_metric_checkpoint_v1
0.82593 / 0.87479 / 0.85012


unet_resnet_152_finetuning_cyclic_lr
0 fold: 0.86494 / 0.89766 / 0.87698
all folds:
    
unet_resnet_152_finetuning_v2_reduce_lr_flip

In [ ]:
# dir_path = os.path.join(MODEL_PATH,'oof')
# os.system("mkdir {}".format(dir_path))

dir_path = os.path.join(MODEL_PATH,'oof')
w='fold_0.25-0.17-0.8067-0.8226.hdf5'
weights_path = os.path.join(MODEL_PATH,w)
model.load_weights(weights_path,by_name=False)

# SAVE OOF PREDICTIONS
pred = predict_test(model=model,
                preds_path=dir_path,
                oof=True,
                ids=ids_valid,
                batch_size=args.batch_size*2,
                thr=0.5,
                TTA='',
                preprocess=preprocess)
res = evaluate([MODEL_PATH], train[train.fold.isin([0])].id.values, 0.5, classification='')
print(np.mean(res['iout']))

In [ ]:
unet_resnet_34_exp_0_resize_144_pad_192_early_stopping
0.79975 / 0.85854 / 0.83097



In [ ]:
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_cycle_lr_div_16_1
fold_0.39-0.19-0.8095-0.8258.hdf5
0.8293827160493826

fold_0.29-0.16-0.7881-0.8075.hdf5
0.8212345679012346

fold_0.37-0.19-0.8130-0.8294.hdf5
0.8325925925925927

In [ ]:
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_checkpoint_cycle_lr
fold_0.27-0.19-0.8004-0.8186.hdf5
0.8218518518518519

fold_0.17-0.16-0.8068-0.8210.hdf5
0.8303703703703704

fold_0.23-0.17-0.8112-0.8298.hdf5
0.8306172839506172

In [ ]:
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_cycle_lr_1_x6
fold_0.17-0.16-0.8068-0.8230.hdf5
0.8288888888888889

fold_0.25-0.17-0.8067-0.8226.hdf5
0.8314814814814815
without pixel deletion: 0.8267901234567901

In [ ]:
best?
fold_0.24-0.15-0.8235-0.8385.hdf5
0.83877 / 0.88339 / 0.85986

fold_0.34-0.16-0.8289-0.8448.hdf5
0.84704 / 0.88595 / 0.86255

fold_0.33-0.16-0.8325-0.8449.hdf5
0.84753 / 0.88669 / 0.86399

fold_0.32-0.16-0.8275-0.8435.hdf5
0.84309 / 0.88288 / 0.85965

fold_0.31-0.16-0.8241-0.8394.hdf5
0.84358 / 0.88345 / 0.85945

fold_0.30-0.16-0.8268-0.8417.hdf5
0.84148 / 0.88327 / 0.85987

fold_0.29-0.16-0.8360-0.8509.hdf5
0.85 / 0.89057 / 0.86772

In [ ]:
MODEL_PATH

In [ ]:
Total params: 25,282,186
Trainable params: 25,264,132
Non-trainable params: 18,054

psp concat:
Total params: 25,283,338
Trainable params: 25,265,284
Non-trainable params: 18,054
    
with UpSampling:
Total params: 28,748,554
Trainable params: 28,730,500
Non-trainable params: 18,054
    
2 conv_blocks in the middle
Total params: 30,005,898
Trainable params: 29,985,796
Non-trainable params: 20,102
    
increase capacity x2:
Total params: 33,003,594
Trainable params: 32,982,852
Non-trainable params: 20,742

In [ ]:
MODEL_PATH

In [ ]:
MODEL_PATH = '/home/branding_images/salt/unet_resnet_50_valid_plus_resize_202_pad_256_bs_24'

In [ ]:
#'/home/branding_images/salt/classification/oof'
res = evaluate([MODEL_PATH], train[train.fold.isin([0,1])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))
res = evaluate([MODEL_PATH], train[train.fold.isin([0])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))
res = evaluate([MODEL_PATH], train[train.fold.isin([1])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
res = evaluate([MODEL_PATH], train[train.fold.isin([0])].id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))


In [ ]:
# res = evaluate(model_pathes, train[train.fold.isin([0])].id.values, 0.5, classification='')
# print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))


In [ ]:
96 -- 3x3
192 -- 6x6
256 -- 8x8
//32
288 -- would be 9x9
384 -- would be 12x12
512 -- would be 16x16

In [ ]:
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last
0.83432 / 0.88136 / 0.8565
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_again
0.84728 / 0.89039 / 0.86695

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle
0.84864 / 0.89183 / 0.86803
    TTA: 0.8521 / 0.89261 / 0.86979
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_again
0.84802 / 0.88714 / 0.8648
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_try
0.84556 / 0.88715 / 0.8643

0.83753 / 0.88699 / 0.86237

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_from_scratch
0.79198 / 0.84075 / 0.81404

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_lr_0001
0.82654 / 0.86517 / 0.84159

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_Conv2DTranspose
0.83728 / 0.88476 / 0.86042
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_Conv2DTranspose_again
0.83802 / 0.88051 / 0.85617

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_2_conv_blocks_in_middle
0.84049 / 0.87981 / 0.85704

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_x2_capacity
0.84074 / 0.88782 / 0.86356

RMSPROP!
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_sgd_0001
0.82802 / 0.86422 / 0.84122

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_sgd_001
0.7579 / 0.82288 / 0.7932

unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_02_do_in_middle_adam_00001
0.83988 / 0.88111 / 0.85803


psp_concat
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_psp_concat
0.84333 / 0.88874 / 0.86519
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_last_psp_concat_again
0.83877 / 0.88339 / 0.85986

    
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_in_middle_96_crop    
0.83667 / 0.8827 / 0.85833    
 
# YOU'VE UPDATED ALBUMENTATIOOONNSS!
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_in_middle_92_crop_resize_101
0.83235 / 0.87777 / 0.85349

# YOU'VE UPDATED ALBUMENTATIOOONNSS!
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_in_middle_96_crop_no_scaling
0.83716 / 0.87899 / 0.85579



# YOU'VE UPDATED ALBUMENTATIOOONNSS!
RESIZE TO 192!
unet_resnet_34_do_exp_0_resize_192_do_04_02_in_middle
0.83086 / 0.87579 / 0.85235
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_checkpoint
0.83593 / 0.87663 / 0.85302
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_checkpoint_again
0.82901 / 0.87291 / 0.84924

# YOU'VE UPDATED ALBUMENTATIOOONNSS!
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_checkpoint_corr
0.8258 / 0.87866 / 0.85393
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_checkpoint_corr_again
0.84432 / 0.8842 / 0.86197
unet_resnet_34_do_exp_0_resize_144_pad_192_do_04_02_middle_checkpoint_revert_albumentations
0.83753 / 0.88699 / 0.86237
< 30 pix: 0.84111 / 0.88865 / 0.8645
*********************************************************
unet_resnet_34_exp_0_resize_144_pad_192
0.82519 / 0.87639 / 0.85106

unet_resnet_34_do_exp_0_resize_144_pad_192
0.83086 / 0.87799 / 0.85441

resnet34_fpn_exp_0_resize_144_pad_192
0.8342 / 0.88145 / 0.85613

unet_resnet_50_exp_0_resize_144_pad_192
0.83531 / 0.88551 / 0.86118

resnet50_fpn_exp_0_resize_144_pad_192
0.84185 / 0.88208 / 0.85874    
*********************************************************
    
unet_resnet_34_do_exp_0_resize_144_pad_192_wtf
0.83333 / 0.88131 / 0.85726
    TTA: 0.8442 / 0.88683 / 0.86383
unet_resnet_34_do_exp_0_resize_144_pad_192_wtf_again
0.84062 / 0.88159 / 0.85864
    TTA: 0.84198 / 0.884 / 0.86134

just on epoch to freeze the encoder (to warm-up decoder a little bit)

In [ ]:
unet_resnet_34_exp_2_resize_144_pad_192 
0.83451 / 0.88055 / 0.857
    fold 0: 0.84716 / 0.88915 / 0.86693
    fold 1: 0.82177 / 0.87189 / 0.847
unet_resnet_34_exp_2_resize_144_pad_192_again
0.83271 / 0.87971 / 0.85611
    fold 0: 0.84222 / 0.88539 / 0.8622
    fold 1: 0.82313 / 0.87398 / 0.84997
        
unet_resnet_34exp_2_resize_144_pad_192_start
0.83538 / 0.87858 / 0.85525
    fold 0: 0.84358 / 0.88594 / 0.86279
    fold 1: 0.82711 / 0.87116 / 0.84765

unet_resnet_34_exp_2_resize_144_pad_192_mask_thr
0.83222 / 0.87609 / 0.85278
    fold 0: 0.84259 / 0.88166 / 0.85915
    fold 1: 0.82177 / 0.87047 / 0.84637
        
unet_resnet_34_do_exp_2_resize_144_pad_192_delete_do -- Deleting DO -- good candidate for the ensemble
0.82999 / 0.87756 / 0.85355
    fold 0: 0.82951 / 0.87334 / 0.84972
    fold 1: 0.83047 / 0.8818 / 0.85741

        
unet_resnet_34_do_exp_2_resize_144_pad_192_do_04_last
0.8381 / 0.88303 / 0.85918
    fold 0: 0.84728 / 0.89413 / 0.86973
    fold 1: 0.82886 / 0.87184 / 0.84856

unet_resnet_34_do_exp_2_resize_144_pad_192_do_04_last_again        
0.83178 / 0.87809 / 0.85378
    0.83395 / 0.87813 / 0.85414
    0.8296 / 0.87804 / 0.85341

unet_resnet_34_do_exp_2_resize_144_pad_192_do_03_last_02_2_prev
0.83216 / 0.88123 / 0.85614
    0.83531 / 0.88476 / 0.85978
    0.82898 / 0.87768 / 0.85247

unet_resnet_34_do_exp_2_resize_144_pad_192_do_02_2_prev
0.82999 / 0.87626 / 0.85208
    0.84963 / 0.88953 / 0.86673
    0.8102 / 0.86289 / 0.83732
    
unet_resnet_34_do_exp_2_resize_144_pad_192_do_02_2_prev_02_last
0.82999 / 0.87628 / 0.85176
    0.84012 / 0.88533 / 0.86138
    0.81978 / 0.86716 / 0.84208
    
unet_resnet_34_do_exp_2_resize_144_pad_192_do_04_last_no_bn
0.81964 / 0.8681 / 0.84299
    0.81889 / 0.86998 / 0.84438
    0.8204 / 0.8662 / 0.84159

unet_resnet_152_exp_0_resize_144_pad_192_do_04_last
0.85531 / 0.89504 / 0.87287

unet_resnet_152_exp_0_resize_144_pad_192_do_04_last_finetune
0.85222 / 0.89078 / 0.86944

unet_resnet_152_exp_0_resize_144_pad_192_do_04_last_lb_metric
0.84667 / 0.88902 / 0.86639
    

In [ ]:
unet_resnet_34_exp_2_resize_192
0.83389 / 0.87669 / 0.8532
    fold 0: 0.84123 / 0.8801 / 0.85735
    fold 1: 0.82649 / 0.87325 / 0.84902
unet_resnet_34_exp_2_resize_192_again
0.83209 / 0.87658 / 0.85316
    fold 0: 0.84506 / 0.88389 / 0.86132
    fold 1: 0.81903 / 0.86922 / 0.84493

unet_resnet_34_exp_2_resize_144_pad_192 
0.83451 / 0.88055 / 0.857
    fold 0: 0.84716 / 0.88915 / 0.86693
    fold 1: 0.82177 / 0.87189 / 0.847
unet_resnet_34_exp_2_resize_144_pad_192_again
0.83271 / 0.87971 / 0.85611
    fold 0: 0.84222 / 0.88539 / 0.8622
    fold 1: 0.82313 / 0.87398 / 0.84997
    
unet_resnet_34_valid_plus_resize_202_pad_256_bs_24
0.8412 / 0.88176 / 0.85929
    fold 0: 0.84901 / 0.88497 / 0.86356
    fold 1: 0.83333 / 0.87852 / 0.85499
unet_resnet_34_exp_2_resize_202_pad_256_bs_24_again
0.83835 / 0.88224 / 0.85882
    fold 0: 0.84691 / 0.88655 / 0.86338
    fold 1: 0.82973 / 0.8779 / 0.85421

unet_resnet_50_exp_2_exp_2_resize_144_pad_192
0.82214 / 0.87442 / 0.85021

unet_resnet_50_valid_plus_resize_202_pad_256_bs_24
0.83848 / 0.88182 / 0.85931

unet_resnet_152_valid_plus_resize_202_pad_256_bs_12_corr
0.85031 / 0.88821 / 0.86668
    0.85716 / 0.89305 / 0.87196
    0.84341 / 0.88334 / 0.86136


In [ ]:
res = evaluate([MODEL_PATH], train.id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
models = ['unet_resnet_34_valid_plus_resize_202_pad_256_bs_24',
'unet_resnet_50_valid_plus_resize_202_pad_256_bs_24',
         'unet_resnet_152_valid_plus_resize_202_pad_256_bs_12_corr','resnet50_fpn_96','unet_resnet_152_finetuning_cyclic_lr'
         ]
model_pathes = ['/home/branding_images/salt/'+x for x in models]
res = evaluate(model_pathes, train.id.values, 0.5, classification='')
print("{} / {} / {}".format(np.round(np.mean(res['iout']),5),np.round(np.mean(res['dice']),5),np.round(np.mean(res['jacard']),5)))

In [ ]:
base 0.85105 / 0.88973 / 0.86871

resnet50_fpn_96
0.85245 / 0.8922 / 0.87103

30 pixels:
0.85262 / 0.89038 / 0.8698

10 pixels:
0.85255 / 0.89154 / 0.87058

In [ ]:
train['iout'] = res['iout']
train.groupby('fold').iout.aggregate('mean')

In [ ]:
MODEL_PATH

In [ ]:
model_pathes

In [ ]:
#pred = ensemble(model_pathes,[0,1,2,3,4],test.id.values,0.5, classification='')
#pred = ensemble([MODEL_PATH],[0,1,2,3,4],test.id.values,0.5, classification='')
#pred = ensemble([MODEL_PATH],[0],test.id.values,0.5, classification='')
pred = ensemble(model_pathes,[0],test.id.values,0.5, classification='')

In [ ]:
pred

In [ ]:
pred

In [ ]:
#test['rle_mask'] = pred
#test[['id','rle_mask']].to_csv('101_102_103_85105.csv',index=False)

In [ ]:
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('unet_resnet_152_exp_0_202_256_scheduler_150_epochs_finetune_0_fold_87272.csv',index=False)

In [ ]:
0.87037 / 0.90118 / 0.88158

In [ ]:
Ensemble 0 fold
0.8763 / 0.9064 / 0.88711

In [ ]:
test['rle_mask'] = pred
test[['id','rle_mask']].to_csv('ensemble_0_fold_88037.csv',index=False)

In [ ]:
test.rle_mask.value_counts(normalize=True)

In [ ]:
train.rle_mask.value_counts(normalize=True,dropna=False)

In [ ]:
def plot2x2Array(image, mask):
    f, axarr = plt.subplots(1,2)
    axarr[0].imshow(image)
    axarr[1].imshow(mask)
    axarr[0].grid()
    axarr[1].grid()
    axarr[0].set_title('Image')
    axarr[1].set_title('Mask')
    
for i in range(5):
    image, mask = dataset[np.random.randint(0, len(dataset))]
    plot2x2Array(image, mask)

In [ ]:
def show_results(idx):
    from rle import rle_decode
    img = rle_decode(pred[idx],(101,101))
    plt.figure(figsize=(4,4))
    plt.imshow(img)
    
    plt.figure(figsize=(4,4))
    plt.imshow(cv2.imread('train/images/{}.png'.format(ids_valid[idx])))
    plt.figure(figsize=(4,4))
    plt.imshow(cv2.imread('train/masks/{}.png'.format(ids_valid[idx])))
    

In [ ]:
for i in range(10):
    show_results(i)

# Classification

In [ ]:
validation_probs=[]
network = 'classification'
folds = [int(f) for f in args.fold.split(',')]
for fold in folds:
    K.clear_session()
    print('***************************** FOLD {} *****************************'.format(fold))

    MODEL_PATH = os.path.join(args.models_dir,network+args.alias)

    if fold == 0:
        if os.path.isdir(MODEL_PATH):
            raise ValueError('Such Model already exists')
        os.system("mkdir {}".format(MODEL_PATH))
        os.system("cp params.py {}".format(MODEL_PATH))

    df_train = train[train.fold != fold].copy().reset_index(drop=True)
    df_valid = train[train.fold == fold].copy().reset_index(drop=True)

    ids_train, ids_valid = df_train.id.values, df_valid.id.values

    print('Training on {} samples'.format(ids_train.shape[0]))
    print('Validating on {} samples'.format(ids_valid.shape[0]))

    # Initialize Model
    weights_path = os.path.join(MODEL_PATH,'fold_{fold}.hdf5'.format(fold=fold))

    print(weights_path.split('/')[-2:])

    model, preprocess = get_model(network, input_shape=(args.input_size, args.input_size, 3), train_base=True)
    print(model.summary())
    model.compile(optimizer=RMSprop(lr=args.learning_rate), loss='binary_crossentropy',
                      metrics=['accuracy'])

    if args.weights is None:
        print('No weights passed, training from scratch')
    else:
        wp = args.weights.format(fold)
        print('Loading weights from {}'.format(wp))
        model.load_weights(wp, by_name=True)

    augs = get_augmentations(args.augmentation_name, p=args.augmentation_prob, input_shape=(args.input_size, args.input_size, 3))


    dg = ClassificationDataGenerator(input_shape=(args.input_size, args.input_size), batch_size = args.batch_size, augs = augs,
                      preprocess = preprocess)

    train_generator = dg.train_batch_generator(ids_train)
    validation_generator = dg.evaluation_batch_generator(ids_valid)

    callbacks = get_callback(args.callback, weights_path=weights_path,
                            fold = fold)

    # Fit the model with Generators:
    model.fit_generator(generator=ThreadsafeIter(train_generator),
                    steps_per_epoch=ids_train.shape[0] // args.batch_size * 2,
                    epochs=args.epochs,
                    callbacks=callbacks,
                    validation_data=ThreadsafeIter(validation_generator),
                    validation_steps=np.ceil(ids_valid.shape[0] / args.batch_size),
                    workers=8)

    model.load_weights(weights_path)

    # SAVE OOF PREDICTIONS
    dir_path = os.path.join(MODEL_PATH,'oof')
    os.system("mkdir {}".format(dir_path))
    pred = classification_predict_test(model=model,
                    preds_path=dir_path,
                    oof=True,
                    ids=ids_valid,
                    batch_size=args.batch_size*4,
                    thr=0.5,
                    TTA='',
                    preprocess=preprocess)
    df_valid['prob'] = pred
    validation_probs.append(df_valid)

    # SAVE TEST PREDICTIONS
    dir_path = os.path.join(MODEL_PATH,'fold_{}'.format(fold))
    os.system("mkdir {}".format(dir_path))
    pred = classification_predict_test(model=model,
                    preds_path=dir_path,
                    oof=False,
                    ids=test.id.values,
                    batch_size=args.batch_size*4,
                    thr=0.5,
                    TTA='',
                    preprocess=preprocess)
    test['prob'] = pred
    test.to_csv(os.path.join(dir_path,'probs_test_fold_{}.csv'.format(fold)),index=False)

    K.clear_session()
    # Run a single fold
    # break

dir_path = os.path.join(MODEL_PATH,'oof')
pd.concat(validation_probs).to_csv(os.path.join(dir_path,'probs_oof.csv'),index=False)